Country: Hungary
Capital: Budapest
Current temperature at capital: 12.4 °C
National currency: HUF
USD/HUF: 356.82 HUF

In [ ]:
BASE_URL1 = "https://restcountries.com/v3.1/name/"

In [ ]:
import requests

In [ ]:
countries = ["Hungary", "Austria", "Czech Republic"]
def get_country_data(country):
  response = requests.get(BASE_URL1 + country)

  if response.status_code == 200:
    data = response.json()[0]
    return {
        "name": data.get("name", "{}").get("common", "N/A"),
        "capital": data.get("capital", "[N/A]"),
        "currency": list(data.get("currencies", {}).keys())[0]
        if "currencies" in data else "N/A",
        "latlng": data.get("capitalInfo", {}).get("latlng", [None, None])
    }
  else:
    print(f"Failed to fetch data for {country}. Status code: {response.status_code}")
    return None

In [ ]:
country_data_list = [get_country_data(country) for country in countries]
for country_data in country_data_list:
  print(country_data)

{'name': 'Hungary', 'capital': ['Budapest'], 'currency': 'HUF', 'latlng': [47.5, 19.08]}
{'name': 'Austria', 'capital': ['Vienna'], 'currency': 'EUR', 'latlng': [48.2, 16.37]}
{'name': 'Czechia', 'capital': ['Prague'], 'currency': 'CZK', 'latlng': [50.08, 14.47]}


In [ ]:
def get_weather_data(lat, lon):
  BASE_URL2 = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m"
  response = requests.get(BASE_URL2.format(lat=lat, lon=lon))

  if response.status_code == 200:
    data = response.json()
    temp = data.get("current", {}).get("temperature_2m", "N/A")
    temp_unit = data.get("current_units", {}).get("temperature_2m", "N/A")
    return {
            str(temp) + temp_unit
    }
  else:
    print(f"Failed to fetch weather data. Status code: {response.status_code}")
    return None

In [ ]:
weather_data_list = [get_weather_data(lat,lon) for (lat,lon) in (country_data.get("latlng") for country_data in country_data_list)]
print(weather_data_list)

[{'7.4°C'}, {'6.2°C'}, {'8.6°C'}]


In [ ]:
BASE_URL3 = "https://open.er-api.com/v6/latest/USD"

In [ ]:
def get_exchange_rate(currency):
  response = requests.get(BASE_URL3)

  if response.status_code == 200:
    data = response.json()
    return data.get("rates", {}).get(currency, "N/A")
  else:
    print(f"Failed to fetch exchange rate data. Status code: {response.status_code}")
    return None

In [ ]:
exchange_data_list = [get_exchange_rate(currency) for currency in (country_data.get("currency") for country_data in country_data_list)]
print(exchange_data_list)

[383.337233, 0.955274, 23.912087]


In [ ]:
for country_data in country_data_list:
  if country_data:
    lat, lon = country_data["latlng"]
    if lat is not None and lon is not None:
      country_data["temperature"] = get_weather_data(lat, lon)
      currency = country_data["currency"]
      if currency != "N/A":
        country_data["exchange_rate"] = get_exchange_rate(currency)
# Display the final dataset
for country_data in country_data_list: print(country_data)

{'name': 'Hungary', 'capital': ['Budapest'], 'currency': 'HUF', 'latlng': [47.5, 19.08], 'temperature': {'7.1°C'}, 'exchange_rate': 383.337233}
{'name': 'Austria', 'capital': ['Vienna'], 'currency': 'EUR', 'latlng': [48.2, 16.37], 'temperature': {'6.2°C'}, 'exchange_rate': 0.955274}
{'name': 'Czechia', 'capital': ['Prague'], 'currency': 'CZK', 'latlng': [50.08, 14.47], 'temperature': {'8.2°C'}, 'exchange_rate': 23.912087}
